In [1]:
import numpy as np
import pickle
from utilities import CtoN
import pandas as pd
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.kernel_approximation import RBFSampler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

model = SGDClassifier(loss='hinge', learning_rate='constant', eta0=0.1, max_iter=2000, tol=1e-3)
# SDGClassifier is an estimator that implements regularized linear models.
# This implementation works with data represented as dense or sparse arrays of floating point 
#    values for the features. The model it fits can be controlled with the loss parameter; by default, 
#    it fits a linear support vector machine.
# For a linear hinge we used 'hinge' for the loss. 
# For learning_rate we used constant which means eta = eta0
# The initial learning rate for the ‘constant’ is 0.1. That is eta=0.1
# The max_iter is teh number of epochs.
# the tol is The stopping criterion.


data_set = pd.read_csv("Reduced Dataset\Guitar Training Dataset PCP.csv") # This reads the dataset and assigns it into an array
dim = data_set.shape # This gives the array dimensions
x = dim[0] # This gives the x dimension = 2000
y = dim[1] - 1 # This gives the y dimension -1 = 13


dim = (x, y) # This creates a new array with x,y dimensions
X = np.zeros(dim) # This assigns zero to all the values
i = 0
while i < 12:
    X[:, i] = data_set[str(i)]
    i += 1

    
y = np.zeros((X.shape)[0])
counter = 0
value = 1
data_per_chord = 200
for i in range(0, (X.shape)[0]):
    if counter == data_per_chord:
        value += 1
        counter = 0
    y[i] = value
    counter += 1
    
    
sampler = AdditiveChi2Sampler()
# This uses sampling the fourier transform of the kernel characteristic at regular intervals.

X = sampler.fit_transform(X)
# This approximates feature map of an RBF kernel by Monte Carlo approximation of its Fourier transform.

model.fit(X, y)
# This users LinearRegression to fit a linear model with coefficients
filename = 'ML_model_ver1.sav'


pickle.dump(model, open(filename, 'wb'))


myModel = pickle.load(open('ML_model_ver1.sav', 'rb'))
#This desirializes the trained model

pred = myModel.predict(X)
print(accuracy_score(pred, y))


0.98


In [ ]:
import numpy as np
import pickle
from utilities import CtoN , NtoC , convert
from PCP import pcp
import os
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.kernel_approximation import RBFSampler

file = str(input("Enter filename: "))

print('file',file)

prev_model='trained_ML_model_ver3.sav'

myModel = pickle.load(open(prev_model, 'rb'))

sampler = AdditiveChi2Sampler()

if file.rsplit('.')[-1]!='wav':
	convert(file)
X = pcp(file)
X = np.array([X])

X = sampler.fit_transform(X)

pred = myModel.predict(X)

print("The model predicted chord to be: ", NtoC(pred[0]))

ans=input("Is the predicted chord correct?[yes|no]\n")
if ans=='yes':
	print('Thanks for using our program.')
else:
	print('We are sorry, please help us train the model further.')
	print('Please enter correct the correct chord')
	t_chord = input("Enter true chord of the wav file: ")
	true_value = np.array([CtoN(t_chord)])
	if true_value != pred:
		myModel.partial_fit(X, true_value)
		pickle.dump(myModel, open(prev_model, 'wb'))
		print('Our model has taken your input into account and corrected itself.')
	else:
		print('Wrong cord.')
        

In [2]:
from utilities import chord_sequence
from pickle import load
from os import listdir
models = [i for i in listdir() if i[-4:] == '.sav']
models.sort()
print('Following models were found. Select which one to use. If you are new, use version 3')
for i in range(len(models)):
    print(i + 1, ':', models[i])
i = int(input('Input model number\n'))
if i > len(models) or i < 1:
    print('Wrong input. Terminating...')
    exit()
model = load(open(models[i - 1], 'rb'))
#file = input("Enter name of file (alongwith path) : ")
file = 'audio\Final_Countdown.wav';
print('The likely chord sequence in audio file is:')
for chord in chord_sequence(model, file, 1):
    print(chord, end = ' ')
print()

Following models were found. Select which one to use. If you are new, use version 3
1 : ML_model_ver1.sav
2 : trained_ML_model_ver3.sav
Input model number
2
The likely chord sequence in audio file is:
File contains Mono channel only. Can't enhance
audio\Final_Countdown.wav


ValueError: X has 36 features per sample; expecting 39